In [1]:
import transformers
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration, LlamaForCausalLM, DPRQuestionEncoder, RagModel, AutoTokenizer, RagTokenForGeneration


c:\Users\XayEss\anaconda3\envs\gpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_dir = "./llama-2-7b-chat-hf"
model = LlamaForCausalLM.from_pretrained(model_dir)

In [3]:
# To load your own indexed dataset built with the datasets library that was saved on disk. More info in examples/rag/use_own_knowledge_dataset.py
from transformers import RagRetriever

dataset_path = "my_dataset"  # dataset saved via *dataset.save_to_disk(...)*
index_path = "my_index.faiss"  # faiss index saved via *dataset.get_index("embeddings").save(...)*

question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
generator_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")


config = transformers.RagConfig(
    question_encoder.config, generator_tokenizer.config,
    index_name="custom",
    dataset_path=dataset_path,
    index_path="my_index.faiss")

retriever = RagRetriever.from_pretrained(
    "facebook/dpr-ctx_encoder-single-nq-base",
    config=config
)

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


AttributeError: 'BartTokenizerFast' object has no attribute 'config'

In [2]:
dataset_path = "my_dataset"  # dataset saved via *dataset.save_to_disk(...)*
index_path = "my_index.faiss"    # faiss index saved via *dataset.get_index("embeddings").save(...)*
retriever = RagRetriever.from_pretrained(
    "facebook/dpr-ctx_encoder-single-nq-base",
    index_name="custom",
    passages_path=dataset_path,
    index_path=index_path,
)

You are using a model of type dpr to instantiate a model of type rag. This is not supported for all configurations of models and can yield errors.


AssertionError: Config has to be initialized with question_encoder and generator config

In [9]:
model = RagModel.from_pretrained("facebook/rag-token-base", retriever=retriever)
tokenizer = AutoTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")



NameError: name 'retriever' is not defined

In [ ]:
rag_config = transformers.RagConfig()
question_encoder = DPRQuestionEncoder.from_pretrained("facebook/rag-token-base", config=rag_config)
tokenizer = RagTokenizer.from_pretrained("facebook/rag-sequence-nq")
retriever = RagRetriever.from_pretrained("facebook/rag-sequence-nq", question_encoder=question_encoder, tokenizer=tokenizer, dataset_path="path/to/my/dataset", index_name="compressed")

rag_model = transformers.RagModel(config=rag_config, generator=model, retriever=retriever, question_encoder=question_encoder)

In [1]:
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration

tokenizer = RagTokenizer.from_pretrained("facebook/rag-sequence-nq")
retriever = RagRetriever.from_pretrained("facebook/rag-sequence-nq", indexed_dataset="wiki_dpr")
model = RagSequenceForGeneration.from_pretrained("facebook/rag-sequence-nq", retriever=retriever)

inputs = tokenizer("What is the capital of France?", return_tensors="pt")
with tokenizer.as_target_tokenizer():
    labels = tokenizer("Paris", return_tensors="pt").input_ids

outputs = model(input_ids=inputs["input_ids"], labels=labels)
loss = outputs.loss
loss.backward()


c:\Users\XayEss\anaconda3\envs\gpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpec

ValueError: Please provide `dataset_path` and `index_path` after calling `dataset.save_to_disk(dataset_path)` and `dataset.get_index('embeddings').save(index_path)`.

In [3]:
from transformers import AutoTokenizer, RagRetriever, RagModel
import torch

tokenizer = AutoTokenizer.from_pretrained("facebook/rag-token-base")
retriever = RagRetriever.from_pretrained(
    "facebook/rag-token-base", index_name="exact", use_dummy_dataset=True
)
# initialize with RagRetriever to do everything in one forward call
model = RagModel.from_pretrained("facebook/rag-token-base", retriever=retriever)



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

In [10]:
inputs = tokenizer("How many people live in Paris?", return_tensors="pt")
outputs = model(input_ids=inputs["input_ids"])

In [11]:
token_ids = torch.argmax(outputs.logits, dim=-1)
decoded_text = tokenizer.decode(token_ids[0], skip_special_tokens=True)
print(decoded_text)

CCa Milà / Barcelona. G building became an exhibition room with an example of modernism in the Eixample. G building is 1,323 m per floor on a plot of 1,620 m. Gaudí made the first sketches in his workshop in the Sagrada Família. He designed the house as a constant curve, both outside and inside, incorporating ruled geometry and naturalistic elements. Casa Milà consists of two buildings, which are structured around two courtyards that provide light to the nine storeys: basement, ground floor, mezzanine, main (or noble) floor, four upper floors, and an attic. The basement was intended to be the garage, the me house many people live in paris? Thep,,,gggibibibibibibibibiberiberiberbumbumiberbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumbumhabibumbumhabibumbumbumbumbumb

In [6]:
import faiss
import numpy as np
from transformers import DPRContextEncoderTokenizer, DPRContextEncoder

class CustomFAISSRetriever:
    def __init__(self, index_path, dataset):
        self.index = faiss.read_index(index_path)
        self.dataset = dataset
        self.tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
        self.encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
    
    def retrieve(self, query, k=5):
        # Encode query
        inputs = self.tokenizer(query, return_tensors="pt", padding=True, truncation=True)
        query_embedding = self.encoder(**inputs).pooler_output.detach().float().numpy()
        
        # Search in FAISS
        _, indices = self.index.search(query_embedding, k)
        
        # Fetch documents
        return [self.dataset[int(idx)]["documents"] for idx in indices[0]]

In [14]:
from datasets import Dataset
dataset_path = "my_dataset"  # dataset saved via *dataset.save_to_disk(...)*
index_path = "my_index.faiss"    # faiss index saved via *dataset.get_index("embeddings").save(...)*
dataset = Dataset.load_from_disk(dataset_path)
print(dataset[0])
retriever = CustomFAISSRetriever(index_path, dataset)
retriever.retrieve("What is a common sight in the forest with the large head?", k=3)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


{'documents': 'The quick brown fox jumps over the lazy dog.', 'embeddings': [-0.2997704744338989, 0.008514147251844406, -0.07928809523582458, 0.13623757660388947, 0.04770223796367645, 0.11831056326627731, 0.09454035013914108, 0.011975111439824104, -0.2180950790643692, -0.5996363162994385, -0.24460801482200623, 0.5288651585578918, -0.12938494980335236, -0.4009478688240051, -0.34350448846817017, 0.098673015832901, 0.3680660128593445, 0.3042343854904175, -0.1828700453042984, -0.8829535841941833, -0.06665748357772827, -0.5842864513397217, -0.3403138816356659, -0.29432517290115356, 0.18226534128189087, 0.2895488739013672, 0.20091497898101807, 0.13872678577899933, -0.13819852471351624, 0.08348187059164047, -0.012238632887601852, -0.22817988693714142, 0.21671785414218903, -0.10647594183683395, -0.04387093335390091, -0.09266112744808197, 0.3513854146003723, -0.5129244923591614, -0.21669374406337738, -0.3910924792289734, 0.11717943847179413, 0.44994306564331055, 0.024473942816257477, 0.60012495

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['The quick brown fox jumps over the lazy dog.',
 'To be, or not to be, that is the question.',
 "Budumba - this is a rare creature that inhabits the forests of the Northern Lands. It is named after the Native Americans who named it 'Budumba' because of its large head. It uses it's head to crack rocks and break up large trees to find food. It is a common sight in the forest."]

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-7b-it",
    torch_dtype=torch.bfloat16,
    max_length=1024,
)



Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 4/4 [00:18<00:00,  4.70s/it]


In [8]:
input_text = "How does Budumba - find it's food?"
result = retriever.retrieve(input_text, k=3)
input_text = "<CONTEXT>" + result[0] + "<QUESTION>" + input_text
input_ids = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

c:\Users\XayEss\anaconda3\envs\gpu\Lib\site-packages\transformers\generation\utils.py:1197: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<bos><CONTEXT>Budumba - this is a rare creature that inhabits the forests of the Northern Lands. It is named after the Native Americans who named it 'Budumba' because of its large head. It uses it's head to crack rocks and break up large trees to find food. It is a common sight in the forest.<QUESTION>How does Budumba - find it's food?

The answer is: Budumba uses its head to crack rocks and break up large trees to find food.<eos>


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
# CPU Enabled uncomment below 👇🏽
# model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")
# GPU Enabled use below 👇🏽
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")